In [11]:
"""
Read dataset file using pandas.
"""

from pandas import read_csv

dataset_path = "./dataset/spotify.csv"

df = read_csv(dataset_path)

In [12]:
"""
Connect to Spotify via spotipy library.
"""

from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import dotenv_values

config = dotenv_values(".env")
CLIENT_ID = config["CLIENT_ID"]
CLIENT_SECRET = config["CLIENT_SECRET"]

auth_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)
sp = Spotify(auth_manager=auth_manager)

In [13]:
from numpy.random import randint

def get_music_infos(num):
    musics_num, _ = df.shape

    music_infos = []

    while len(music_infos) < num:
        play_music_index = randint(low=0, high=musics_num)

        music = df.iloc[play_music_index, :]

        music_info = get_music_info(music)
        preview_url = music_info["preview_url"]

        if preview_url != None:
            music_infos.append(music_info)

    return music_infos


def get_music_info(music):
    music_id = music["id"]

    track = sp.track(track_id=music_id)

    name = track["name"]
    artists = track["artists"]
    artist_names = map(lambda x: x["name"], artists)
    preview_url = track["preview_url"]

    music_info = {
        "name": name,
        "artist_names": artist_names,
        "preview_url": preview_url,
    }

    return music_info


In [14]:
from textwrap import dedent
from vlc import MediaPlayer

def print_song_info(no, name, artist_names):
	artist_name = " & ".join(artist_names)

	print(dedent(f"""
		---
		Song No.{no}
		Now Playing
				
		Name: {name}
		Artist: {artist_name}
		---
	"""))

def get_user_input():
	user_input = str(input(dedent(f"""
		Do you enjoy this music ?

		If yes, press (y).
		If no, press (n).

		Press (s) to stop/pause the music.
		Press (p) to play/resume the music.
		Press (e) to exit.
	""")))
	user_input_lowercase = user_input.lower()

	if user_input_lowercase in ["s", "p", "y", "n", "e"]:
		return user_input_lowercase
	else:
		return get_user_input()

In [15]:
liked_music_infos = []
disliked_music_infos = []

music_infos = get_music_infos(num=10)

for index, music_info in enumerate(music_infos):
	no = index + 1
	name = music_info["name"]
	artist_names = music_info["artist_names"]
	preview_url = music_info["preview_url"]

	print_song_info(no, name, artist_names)

	media_player = MediaPlayer(preview_url)
	media_player.play()

	is_finish = False
	is_exit = False

	while(is_finish != True):
		user_input = get_user_input()

		if user_input == "e":
			media_player.stop()

			is_finish = True
			is_exit = True
		elif user_input == "s":
			media_player.set_pause(True)
		elif user_input == "p":
			is_media_playing = media_player.is_playing()

			if is_media_playing:
				media_player.set_pause(False)
			else:
				media_player.play()
		elif user_input == "y":
			media_player.stop()
			is_finish = True

			liked_music_infos.append(music_info)
		elif user_input == "n":
			media_player.stop()
			is_finish = True

			disliked_music_infos.append(music_info)
	
	if is_exit:
		break


---
Song No.1
Now Playing

Name: Fill the Space
Artist: Stars Of Stage And Screen
---


---
Song No.2
Now Playing

Name: Stahovavi vtaci
Artist: Richard Muller
---


---
Song No.3
Now Playing

Name: Espasmo Fulgor
Artist: Rogelio Sosa & Irvine Arditti
---


---
Song No.4
Now Playing

Name: bad batman
Artist: Deli Girls
---


---
Song No.5
Now Playing

Name: The Blue Kite
Artist: Xuefei Yang
---


---
Song No.6
Now Playing

Name: Give Back the Key to My Heart
Artist: Uncle Tupelo
---


---
Song No.7
Now Playing

Name: Addams Family Pinball
Artist: Network Nick
---


---
Song No.8
Now Playing

Name: Gta3
Artist: MC Hawking
---


---
Song No.9
Now Playing

Name: My Life of Crime
Artist: PARKAS
---


---
Song No.10
Now Playing

Name: Howudoin?
Artist: Shammy Dee
---



[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 123, prop: [atfp], OSStatus: 1852797029
[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 99, prop: [atfp], OSStatus: 1852797029
[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 113, prop: [atfp], OSStatus: 1852797029
[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 99, prop: [atfp], OSStatus: 1852797029
[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 113, prop: [atfp], OSStatus: 1852797029
[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 99, prop: [atfp], OSStatus: 1852797029
[00007fbb7b4a58c0] auhal audio output error: AudioObjectAddPropertyListener failed, device id 113, prop: [atfp], OSStatus: 1852797029
